In [40]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
%cd /content/drive/My Drive/Colab Notebooks/adversarial/adversarial_reisz
%cp -R * /content
%cd /content

/content/drive/My Drive/Colab Notebooks/adversarial/adversarial_reisz
/content


In [42]:
%load_ext autoreload
%autoreload 2

[autoreload of advreisz.deepreisz.oadam failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: step() requires a code object with 1 free v

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
!pip install econml

In [44]:
!python setup.py develop

running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.



In [45]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scipy
import scipy.special
from sklearn.linear_model import LassoCV, LogisticRegressionCV, LinearRegression, Lasso, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
import torch
import torch.nn as nn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.utils.multiclass import type_of_target

from debiased import DebiasedMoment
from advreisz.linear import SparseLinearAdvRiesz
from advreisz.kernel import AdvNystromKernelReisz, AdvKernelReisz, NystromKernelReisz, KernelReisz
from advreisz.deepreisz import AdvReisz
from advreisz.ensemble import AdvEnsembleReisz, RFrr, interactive_poly_feature_fns
from utilities import AutoKernel, prod_kernel, PluginRR2, FitParamsWrapper

# Definition of Diff-in-Diff Moment

In [46]:
# E[E[Y|D=1, A=1, X] – E[Y|D=0, A=1, X] – (E[Y|D=1, A=0, X] – E[Y|D=0, A=0, X])]
# D is the first column, A is the second, and X is the remaining columns.
def moment_fn(x, test_fn):
    n_obs = x.shape[0]
    if torch.is_tensor(x):
        with torch.no_grad():
            t11 = torch.cat([torch.ones((n_obs, 2)).to(device), x[:, 2:]], dim=1)
            t01 = torch.cat([torch.zeros((n_obs, 1)).to(device), torch.ones((n_obs, 1)).to(device), x[:, 2:]], dim=1)
            t10 = torch.cat([torch.ones((n_obs, 1)).to(device), torch.zeros((n_obs, 1)).to(device), x[:, 2:]], dim=1)
            t00 = torch.cat([torch.zeros((n_obs, 2)).to(device), x[:, 2:]], dim=1)
    else:
        t11 = np.hstack([np.ones((n_obs, 2)), x[:, 2:]])
        t01 = np.hstack([np.zeros((n_obs, 1)), np.ones((n_obs, 1)), x[:, 2:]])
        t10 = np.hstack([np.ones((n_obs, 1)), np.zeros((n_obs, 1)), x[:, 2:]])
        t00 = np.hstack([np.zeros((n_obs, 2)), x[:, 2:]])
    return test_fn(t11) - test_fn(t01) - test_fn(t10) + test_fn(t00)

# Functions for Different Adversarial Riesz Estimators

In [47]:
def get_reg_fn(X, y):
    est = Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                    ('s', StandardScaler()),
                    ('lasso', LassoCV(max_iter=10000, random_state=123))])
    est.fit(X, y)

    return lambda: Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                             ('s', StandardScaler()),
                             ('lasso', Lasso(alpha=est.named_steps['lasso'].alpha_, max_iter=10000, random_state=123))])


def get_splin_fn(X):
    return lambda: SparseLinearAdvRiesz(moment_fn,
                                        featurizer=Pipeline([('p', PolynomialFeatures(degree=2, include_bias=False)),
                                                             ('s', StandardScaler()),
                                                             ('cnt', PolynomialFeatures(degree=1, include_bias=True))]),
                                        n_iter=50000, lambda_theta=0.01, B=10,
                                        tol=0.00001)


def get_advnyskernel_fn(X):
    est = AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                regm='auto', regl='auto', n_components=100, random_state=123)
    reg = est.opt_reg(X)
    return lambda: AdvNystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                         regm=6*reg, regl=reg, n_components=100, random_state=123)


def get_advkernel_fn(X):
    est = AdvKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regm='auto', regl='auto')
    reg = est.opt_reg(X)
    return lambda: AdvKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regm=6*reg, regl=reg)


def get_nyskernel_fn(X):
    est = NystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                             regl='auto', n_components=100, random_state=123)
    reg = est.opt_reg(X)
    return lambda: NystromKernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]),
                                      regl=reg, n_components=100, random_state=123)


def get_kernel_fn(X):
    est = KernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regl='auto')
    reg = est.opt_reg(X)
    return lambda: KernelReisz(kernel=lambda X, Y=None: prod_kernel(X, Y=Y, gamma=1.0/X.shape[1]), regl=reg)



def get_lg_plugin_fn(X):
    clf = LogisticRegressionCV(cv=3, max_iter=10000, random_state=123)
    C_ = clf.fit(X[:, 2:], X[:, 1]).C_[0]
    model_t_A = LogisticRegression(C=C_, max_iter=10000, random_state=123)
    clf = LogisticRegressionCV(cv=3, max_iter=10000, random_state=123)
    C_ = clf.fit(X[:, 1:], X[:, 0]).C_[0]
    model_t_treat = LogisticRegression(C=C_, max_iter=10000, random_state=123)
    return lambda: PluginRR2(model_t_A=model_t_A, model_t_treat=model_t_treat,
                             min_propensity=1e-6)


def get_rf_plugin_fn(X):
    gcv = GridSearchCV(RandomForestClassifier(bootstrap=True, random_state=123),
                       param_grid={'max_depth': [3, None],
                                   'min_samples_leaf': [10, 50]},
                       scoring='r2',
                       cv=5)
    best_model_A = clone(gcv.fit(X[:, 2:], X[:, 1]).best_estimator_)
    best_model_treat = clone(clone(gcv).fit(X[:, 1:], X[:, 0]).best_estimator_)
    return lambda: PluginRR2(model_t_A=best_model_A,
                             model_t_treat=best_model_treat,
                             min_propensity=1e-6)


device = torch.cuda.current_device() if torch.cuda.is_available() else None

# Returns a deep model for the reisz representer
def get_learner(n_t, n_hidden, p):
    return nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_hidden), nn.LeakyReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.LeakyReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, 1))

# Returns a deep model for the test functions
def get_adversary(n_z, n_hidden, p):
    return nn.Sequential(nn.Dropout(p=p), nn.Linear(n_z, n_hidden), nn.ReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ReLU(),
                         nn.Dropout(p=p), nn.Linear(n_hidden, 1))

print("GPU:", torch.cuda.is_available())

def get_agmm_fn(X):
    n_hidden = 100
    dropout = 0.5
    return lambda: FitParamsWrapper(AdvReisz(get_learner(X.shape[1], n_hidden, dropout),  # Edited so that #features in is #cols of X
                                             get_adversary(X.shape[1], n_hidden, dropout),
                                             moment_fn),
                                   val_fr=.2,
                                   preprocess_epochs=200,
                                   earlystop_rounds=100,
                                   store_test_every=20,
                                   learner_lr=1e-4, adversary_lr=1e-4,
                                   learner_l2=6e-4, adversary_l2=1e-4,
                                   n_epochs=1000, bs=100,
                                   logger=None, model_dir=str(Path.home()), device=device, verbose=1)

GPU: True


# Hyperparameters

In [48]:
n_splits = 1
res = {}
q=0  # Data not split up into quantiles
res[f'q={q}'] = {}
print(f'Quintile={q}')

Quintile=0


# Load Data

In [49]:
# get data
# df = pd.read_stata('../charitable_giving/Replication/AER merged.dta', convert_categoricals=False)
df = pd.read_stata('https://github.com/gsbDBI/ExperimentData/raw/master/Charitable/RawData/AER%20merged.dta',
                   convert_categoricals=False)
df = df.loc[(df['ratio'] == 0) | (df['ratio'] == 1)]
df = df.drop(['control', 'ratio', 'ratio2', 'ratio3',
            'size', 'size25', 'size50', 'size100', 'sizeno',
            'ask', 'askd1', 'askd2', 'askd3', 'ask1', 'ask2', 'ask3',
            'gave', 'amountchange', 'state50one', 'blue0'], axis=1)
# state50one just tags one (arbitrary?) observation for each state
# blue0 and red0 and perfectly collinear (when all variables are nonmissing); bluecty and redcty are not
df = df.dropna()
y = df['amount'].values
X = df[['treatment', 'red0',
        'hpa', 'year5', 'dormant', 'nonlit', 'cases', 'redcty', 'bluecty',
        'pwhite', 'pblack', 'page18_39', 'ave_hh_sz', 'median_hhincome', 'powner', 'psch_atlstba', 'pop_propurban']].values

# df = pd.read_stata('../charitable_giving/Replication/AER merged.dta', convert_categoricals=False)
# df = df.loc[(df['ratio'] == 0) | (df['ratio'] == 1)]
# df = df.drop(['control', 'ratio', 'ratio2', 'ratio3',
#             'size', 'size25', 'size50', 'size100', 'sizeno',
#             'ask', 'askd1', 'askd2', 'askd3', 'ask1', 'ask2', 'ask3',
#             'gave', 'amountchange', 'state50one', 'blue0'], axis=1)
# # state50one just tags one (arbitrary?) observation for each state
# # blue0 and red0 and perfectly collinear (when all variables are nonmissing); bluecty and redcty are not
# df = df.dropna()
# y = df['amount'].values
# X = df[['treatment', 'red0',
#         'hpa', 'year5', 'dormant', 'nonlit', 'cases', 'perbush', 'redcty', 'bluecty',
#         'pwhite', 'pblack', 'page18_39', 'ave_hh_sz', 'median_hhincome', 'powner', 'psch_atlstba', 'pop_propurban']].values

In [50]:
# scale non-binary variables
y = y.astype(np.double)
X = X.astype(np.double)
idx_nonbi = [i for i in range(2, X.shape[1]) if type_of_target(X[:, i]) != 'binary']  # indices of non-binary variables (first and second columns should be binary)
X[:, idx_nonbi] = StandardScaler().fit_transform(X[:, idx_nonbi])
y_scale = np.std(y)
y = y / y_scale

# shuffle data
inds = np.arange(X.shape[0])
np.random.seed(123)
np.random.shuffle(inds)
X, y = X[inds].copy(), y[inds].copy()

# filter extreme party and treatment propensities
clf_party = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 2:], X[:, 1])
clf_treat = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 1:], X[:, 0])
prop_party = clf_party.predict_proba(X[:, 2:])[:, 1]
prop_treat = clf_treat.predict_proba(X[:, 1:])[:, 1]
filt = (prop_party <= .9) & (prop_party >= .1) & (prop_treat <= .9) & (prop_treat >= .1)
print(X.shape[0], np.sum(filt))
X, y = X[filt], y[filt]

25859 21712


# Adversarial RF

In [ ]:
def get_rf_fn(X):
    return lambda: AdvEnsembleReisz(moment_fn=moment_fn,
                                    n_treatments=2,
                                    max_abs_value=100,
                                    n_iter=400, degree=1)

est = DebiasedMoment(moment_fn=moment_fn,
                     get_reisz_fn=get_rf_fn,
                     get_reg_fn=get_reg_fn,
                     n_splits=1)
est.fit(X, y)
p, s, l, u = est.avg_moment()
{'point': p * y_scale, 'stderr': s * y_scale, 'lower': l * y_scale, 'upper': u * y_scale}

# Try All Methods

In [ ]:
n_splits = 1
res = {}
q=0  # Data not split up into quantiles
res[f'q={q}'] = {}
print(f'Quintile={q}')

for name, get_reisz_fn in [
                            ('splin', get_splin_fn),
                            # ('advrkhs', get_advkernel_fn),
                            # # ('rkhs', get_kernel_fn),
                            ('nys_advrkhs', get_advnyskernel_fn),
                            # # ('nys_rkhs', get_nyskernel_fn),
                            ('plugin_lg', get_lg_plugin_fn),
                            ('plugin_rf', get_rf_plugin_fn),
                            ('advnnet', get_agmm_fn),
                            ('advrf', get_rf_fn)
                            ]:
    est = DebiasedMoment(moment_fn=moment_fn,
                            get_reisz_fn=get_reisz_fn,
                            get_reg_fn=get_reg_fn, n_splits=n_splits)
    est.fit(X, y)
    p, s, l, u = est.avg_moment()
    res[f'q={q}'][name] = {'point': p * y_scale, 'stderr': s * y_scale,
                            'lower': l * y_scale, 'upper': u * y_scale}
    p, s, l, u = est.avg_moment(tmle=True)
    res[f'q={q}'][f'{name}_tmle'] = {'point': p * y_scale, 'stderr': s * y_scale,
                                        'lower': l * y_scale, 'upper': u * y_scale}
    print(name, res[f'q={q}'][name])

res[f'q={q}'] = pd.DataFrame(res[f'q={q}']).transpose()
res = pd.concat(res)

Quintile=0


/content/advreisz/linear/sparse_linear.py:175: UserWarning: Maximum number of iterations reached and duality gap tolerance not achieved. Consider increasing the maximum number of iterations.
  warnings.warn("Maximum number of iterations reached and duality gap tolerance not achieved. "


splin {'point': 0.36508933469909116, 'stderr': 0.1988587410628784, 'lower': -0.024688365458599728, 'upper': 0.7548670348567821}
nys_advrkhs {'point': 0.17149374721128846, 'stderr': 0.11763264276928773, 'lower': -0.0590748499598382, 'upper': 0.4020623443824151}
plugin_lg {'point': 0.40866032749901765, 'stderr': 0.3491947213687799, 'lower': -0.27578690713016935, 'upper': 1.0931075621282047}
plugin_rf {'point': 0.19566232975305275, 'stderr': 0.1411899316438051, 'lower': -0.08108027933418267, 'upper': 0.4724049388402882}
Splitting
Epoch #0
Epoch #1
Epoch #2
Epoch #3
Epoch #4
Epoch #5
Epoch #6
Epoch #7
Epoch #8
Epoch #9
Epoch #10
Epoch #11
Epoch #12
Epoch #13
Epoch #14
Epoch #15
Epoch #16
Epoch #17
Epoch #18
Epoch #19
Epoch #20
Epoch #21
Epoch #22
Epoch #23
Epoch #24
Epoch #25
Epoch #26
Epoch #27
Epoch #28
Epoch #29
Epoch #30
Epoch #31
Epoch #32
Epoch #33
Epoch #34
Epoch #35
Epoch #36
Epoch #37
Epoch #38
Epoch #39
Epoch #40
Epoch #41
Epoch #42
Epoch #43
Epoch #44
Epoch #45
Epoch #46
Epoch #

In [ ]:
res

point     stderr       lower       upper
q=0 splin               0.365089   0.198859   -0.024688    0.754867
    splin_tmle          0.459150   0.198812    0.069463    0.848836
    nys_advrkhs         0.171494   0.117633   -0.059075    0.402062
    nys_advrkhs_tmle    0.064036   0.117634   -0.166534    0.294607
    plugin_lg           0.408660   0.349195   -0.275787    1.093108
    plugin_lg_tmle      0.391040   0.349594   -0.294189    1.076269
    plugin_rf           0.195662   0.141190   -0.081080    0.472405
    plugin_rf_tmle    599.852578  50.281139  501.297862  698.407294
    advnnet             0.326141   0.200125   -0.066119    0.718402
    advnnet_tmle        0.418931   0.200102    0.026716    0.811146
    advrf               0.168707   0.058987    0.053088    0.284327
    advrf_tmle          0.075842   0.058950   -0.039704    0.191388

In [ ]:
import joblib
joblib.dump(res, '/content/drive/My Drive/Colab Notebooks/adversarial/adversarial_reisz/results_charitable.jbl')

['/content/drive/My Drive/Colab Notebooks/adversarial/adversarial_reisz/results_charitable.jbl']

In [51]:
import joblib
res = joblib.load('/content/drive/My Drive/Colab Notebooks/adversarial/adversarial_reisz/results_charitable.jbl')

In [69]:
res.iloc[[not s.endswith('_tmle') for s in list(res.index.get_level_values(1))]]

point    stderr     lower     upper
q=0 splin        0.365089  0.198859 -0.024688  0.754867
    nys_advrkhs  0.171494  0.117633 -0.059075  0.402062
    plugin_lg    0.408660  0.349195 -0.275787  1.093108
    plugin_rf    0.195662  0.141190 -0.081080  0.472405
    advnnet      0.326141  0.200125 -0.066119  0.718402
    advrf        0.168707  0.058987  0.053088  0.284327

# Test Adversarial RF on 401k Data

In [ ]:
def moment_fn(x, test_fn):
    n_obs = x.shape[0]
    t1 = np.hstack([np.ones((n_obs, 1)), x[:, 1:]])
    t0 = np.hstack([np.zeros((n_obs, 1)), x[:, 1:]])
    return test_fn(t1) - test_fn(t0)

def get_rf_fn(X):
    return lambda: AdvEnsembleReisz(moment_fn=moment_fn,
                                    n_treatments=1,
                                    max_abs_value=15,
                                    degree=1)

# from advreisz.ensemble import AdvEnsembleReisz

# def get_rf_fn(X):
#     return lambda: AdvEnsembleReisz(moment_fn=moment_fn, max_abs_value=15, degree=1)

df = pd.read_csv(f'401k/quintile0_trimmed.csv', index_col=0)
y = df['Y'].values
X = df[['D'] + [f'X{i}' for i in np.arange(1, 10)]].values

# scale data
y = y.astype(np.double)
X = X.astype(np.double)
X[:, 1:5] = StandardScaler().fit_transform(X[:, 1:5])
y_scale = np.std(y)
y = y / y_scale

# shuffle data
inds = np.arange(X.shape[0])
np.random.seed(123)
np.random.shuffle(inds)
X, y = X[inds].copy(), y[inds].copy()

# filter extrement propensities
clf = LogisticRegressionCV(cv=5, max_iter=10000, random_state=123).fit(X[:, 1:], X[:, 0])
prop = clf.predict_proba(X[:, 1:])
filt = (prop[:, 1] <= .9) & (prop[:, 1] >= .1)
print(X.shape[0], np.sum(filt))
X, y = X[filt], y[filt]

est = DebiasedMoment(moment_fn=moment_fn,
                     get_reisz_fn=get_rf_fn,
                     get_reg_fn=get_reg_fn, n_splits=1)
est.fit(X, y)
p, s, l, u = est.avg_moment()
{'point': p * y_scale, 'stderr': s * y_scale, 'lower': l * y_scale, 'upper': u * y_scale}

9848 9804


{'point': 7489.894500144199,
 'stderr': 1116.3064149611787,
 'lower': 5301.703958403848,
 'upper': 9678.085041884551}

In [ ]:
n_splits = 1
res = {}
q=0  # Data not split up into quantiles
res[f'q={q}'] = {}
print(f'Quintile={q}')

for name, get_reisz_fn in [
                            ('splin', get_splin_fn),
                            # ('advrkhs', get_advkernel_fn),
                            # # ('rkhs', get_kernel_fn),
                            ('nys_advrkhs', get_advnyskernel_fn),
                            # # ('nys_rkhs', get_nyskernel_fn),
                            # ('advnnet', get_agmm_fn),
                            ('advrf', get_rf_fn)
                            ]:
    est = DebiasedMoment(moment_fn=moment_fn,
                            get_reisz_fn=get_reisz_fn,
                            get_reg_fn=get_reg_fn, n_splits=n_splits)
    est.fit(X, y)
    p, s, l, u = est.avg_moment()
    res[f'q={q}'][name] = {'point': p * y_scale, 'stderr': s * y_scale,
                            'lower': l * y_scale, 'upper': u * y_scale}
    p, s, l, u = est.avg_moment(tmle=True)
    res[f'q={q}'][f'{name}_tmle'] = {'point': p * y_scale, 'stderr': s * y_scale,
                                        'lower': l * y_scale, 'upper': u * y_scale}
    print(name, res[f'q={q}'][name])

print(res[f'q={q}'])
res[f'q={q}'] = pd.DataFrame(res[f'q={q}']).transpose()
res = pd.concat(res)

Quintile=0


C:\Users\bsyrg\Documents\adversarial\adversarial_reisz\advreisz\linear\sparse_linear.py:175: UserWarning: Maximum number of iterations reached and duality gap tolerance not achieved. Consider increasing the maximum number of iterations.
  warnings.warn("Maximum number of iterations reached and duality gap tolerance not achieved. "


splin {'point': 7797.382835248444, 'stderr': 1040.2336841988708, 'lower': 5758.310517418176, 'upper': 9836.455153078712}
nys_advrkhs {'point': 7642.738472013803, 'stderr': 1123.2507232153064, 'lower': 5440.93565551004, 'upper': 9844.541288517566}
advrf {'point': 7489.894500144199, 'stderr': 1116.3064149611787, 'lower': 5301.703958403848, 'upper': 9678.085041884551}
{'splin': {'point': 7797.382835248444, 'stderr': 1040.2336841988708, 'lower': 5758.310517418176, 'upper': 9836.455153078712}, 'splin_tmle': {'point': 7807.1941498693905, 'stderr': 1040.4668854910922, 'lower': 5767.664709464961, 'upper': 9846.723590273821}, 'nys_advrkhs': {'point': 7642.738472013803, 'stderr': 1123.2507232153064, 'lower': 5440.93565551004, 'upper': 9844.541288517566}, 'nys_advrkhs_tmle': {'point': 7644.322691514144, 'stderr': 1123.5401525565867, 'lower': 5441.952533876617, 'upper': 9846.69284915167}, 'advrf': {'point': 7489.894500144199, 'stderr': 1116.3064149611787, 'lower': 5301.703958403848, 'upper': 9678.

In [ ]:
res

point       stderr        lower        upper
q=0 splin             7797.382835  1040.233684  5758.310517  9836.455153
    splin_tmle        7807.194150  1040.466885  5767.664709  9846.723590
    nys_advrkhs       7642.738472  1123.250723  5440.935656  9844.541289
    nys_advrkhs_tmle  7644.322692  1123.540153  5441.952534  9846.692849
    advrf             7489.894500  1116.306415  5301.703958  9678.085042
    advrf_tmle        7502.701410  1116.039344  5315.034383  9690.368437